# Анализ базы данных сервиса для чтения книг (SQL).

Компания решила купить крупный сервис для чтения книг по подписке.

Задача проекта — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


Описание данных 

Таблица books

Содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц; 
- publication_date — дата публикации книги; 
- publisher_id — идентификатор издателя.

Таблица authors

Содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers

Содержит данные об издательствах:

- publisher_id — идентификатор издательства;
- publisher — название издательства; 

Таблица ratings

Содержит данные о пользовательских оценках книг: 

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку; 
- rating — оценка книги.

Таблица reviews

Содержит данные о пользовательских обзорах на книги:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор; 
- text — текст обзора.

### Подключение к базе данных.

In [1]:
!pip install sqlalchemy 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine 
import seaborn as sns


In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Анализ базы данных.

Выведим первые строки каждой таблицы и создадим первое представление о базе.

In [4]:
query = '''
SELECT *
FROM books
LIMIT 1;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [5]:
query = '''
SELECT *
FROM publishers
LIMIT 1;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,publisher_id,publisher
0,1,Ace


In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 1;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 1;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


#### 

- Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(*)
FROM books as b
WHERE b.publication_date > '2000-01-01'
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,count
0,819


То есть после 1 января 2000 года было выпущено 819 книг.

#### 

- Для каждой книги посчитаем количество обзоров и среднюю оценку;


In [9]:
#выведим таблицу books
query = '''
SELECT *
       
FROM books as b

;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [10]:
#выведем в таблице ratings среднюю оценку по каждой книге
query = '''
SELECT book_id,
       AVG(r.rating)
       
FROM ratings as r
GROUP BY r.book_id
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,book_id,avg
0,652,4.500000
1,273,4.500000
2,51,4.250000
3,951,4.000000
4,839,4.285714
...,...,...
995,64,4.230769
996,55,5.000000
997,148,3.428571
998,790,3.500000


In [11]:
#в таблице reviews посчитаем количество обзоров по каждой книге
query = '''
SELECT rv.book_id,
       COUNT(review_id)
       
FROM reviews as rv
GROUP BY rv.book_id
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,book_id,count
0,652,2
1,273,2
2,51,5
3,951,2
4,839,4
...,...,...
989,64,4
990,55,2
991,148,3
992,790,2


In [12]:
# объединим три таблицы 
query = '''
WITH 
avg_r AS  
(SELECT book_id,
       AVG(r.rating)
FROM ratings as r
GROUP BY r.book_id),

count_rv AS 
(SELECT rv.book_id,
        COUNT(review_id)
FROM reviews as rv
GROUP BY rv.book_id) 
       
SELECT *
FROM books as b
LEFT JOIN avg_r ON avg_r.book_id = b.book_id
LEFT JOIN count_rv ON count_rv.book_id = b.book_id       

;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,book_id,author_id,title,num_pages,publication_date,publisher_id,book_id,avg,book_id,count
0,652,3,The Body in the Library (Miss Marple #3),191,2006-09-01,50,652,4.500000,652.0,2.0
1,273,352,Galápagos,324,1999-01-12,88,273,4.500000,273.0,2.0
2,51,54,A Tree Grows in Brooklyn,496,2006-05-30,135,51,4.250000,51.0,5.0
3,951,540,Undaunted Courage: The Pioneering First Missio...,592,2003-10-06,269,951,4.000000,951.0,2.0
4,839,332,The Prophet,127,2010-01-01,251,839,4.285714,839.0,4.0
...,...,...,...,...,...,...,...,...,...,...
995,672,144,The Cat in the Hat and Other Dr. Seuss Favorites,61,2003-10-14,174,672,5.000000,NaN,NaN
996,83,174,Anne Rice's The Vampire Lestat: A Graphic Novel,404,1991-11-30,33,83,3.666667,NaN,NaN
997,221,151,Essential Tales and Poems,688,2004-10-25,42,221,4.000000,NaN,NaN
998,387,365,Leonardo's Notebooks,352,2005-08-01,49,387,4.000000,NaN,NaN


In [13]:
#найдем в объединенной таблице необходимые значения по каждой книге
query = '''
WITH 
avg_r AS  
(SELECT book_id,
       AVG(r.rating)
FROM ratings as r
GROUP BY r.book_id),

count_rv AS 
(SELECT rv.book_id,
        COUNT(review_id)
FROM reviews as rv
GROUP BY rv.book_id) 
       
SELECT b.title,
       ROUND(AVG(avg_r.avg), 2),
       SUM(count)
FROM books as b
LEFT JOIN avg_r ON avg_r.book_id = b.book_id
LEFT JOIN count_rv ON count_rv.book_id = b.book_id       
GROUP BY b.book_id
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,title,round,sum
0,The Body in the Library (Miss Marple #3),4.50,2.0
1,Galápagos,4.50,2.0
2,A Tree Grows in Brooklyn,4.25,5.0
3,Undaunted Courage: The Pioneering First Missio...,4.00,2.0
4,American Gods (American Gods #1),4.00,5.0
...,...,...,...
995,Possession,4.00,3.0
996,A Woman of Substance (Emma Harte Saga #1),5.00,2.0
997,Christine,3.43,3.0
998,The Magicians' Guild (Black Magician Trilogy #1),3.50,2.0


По каждой книге посчитали количество обзоров и среднюю оценку. Видим, что количество обзоров не влияет на среднюю оценку пользователей.

#### 

- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [14]:
query = '''
SELECT pub.publisher,
       b.publisher_id,
       COUNT(b.book_id)
         
       
       
FROM books as b
LEFT JOIN publishers as pub ON pub.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY b.publisher_id, pub.publisher 
ORDER BY COUNT(b.book_id) DESC
LIMIT 1


         
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,publisher,publisher_id,count
0,Penguin Books,212,42


Определили издателя - Penguin Books, которое выпустило наибольшее число книг толще 50 страниц.

#### 

- Определим автора с самой высокой средней оценкой книг (только книги с 50 и более оценками)

In [15]:
query = '''
SELECT t.author,
       t.author_id,
       AVG(t.avg_rat) as avg_rat
FROM (SELECT aut.author,
       b.author_id, 
       AVG(r.rating) as avg_rat,
       COUNT(r.rating_id) as count_rat
FROM books as b
LEFT JOIN ratings as r ON r.book_id = b.book_id
LEFT JOIN authors as aut ON aut.author_id = b.author_id
GROUP BY b.book_id, b.author_id, aut.author) as t
WHERE t.count_rat > 50
GROUP BY t.author_id, t.author 
ORDER BY avg_rat DESC
LIMIT 1;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,author,author_id,avg_rat
0,J.K. Rowling/Mary GrandPré,236,4.283844


Определили идентификатор автора с самой высокой средней оценкой книг (только книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré.

#### 

- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [19]:
query = '''
WITH 
r AS 
(SELECT r.username,
       COUNT(r.rating)

FROM ratings as r
GROUP BY r.username
HAVING (COUNT(r.rating)) > 50) 

SELECT ROUND(AVG(rv.review_id),0)
FROM reviews as rv
INNER JOIN r ON r.username = rv.username
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,count
0,146


То есть среднее количество обзоров пользователей, которые поставили больше 50 оценок равно 1316 обзоров. 

In [25]:
query = '''
WITH 
r AS 
(SELECT r.username,
       COUNT(r.rating)

FROM ratings as r
GROUP BY r.username
HAVING (COUNT(r.rating)) > 50) 

SELECT COUNT(rv.review_id),
        rv.username
FROM reviews as rv
INNER JOIN r ON r.username = rv.username
GROUP BY rv.username
;
'''
sample_df = pd.io.sql.read_sql(query, con = engine)
sample_df

,count,username
0,28,sfitzgerald
1,25,jennifermiller
2,18,xdavis
3,22,paul88
4,27,martinadam
5,26,richard89
